In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import os
import pickle as pkl
from tqdm import tqdm

In [3]:
path = os.getcwd()
data_dir = path + '/data/'
print(data_dir)

C:\Users\atulg\FairFrame/data/


## Read in Reviews

In [4]:
all_reviews = pkl.load(open(data_dir + 'all_reviews.p','rb'))
all_reviews = all_reviews[1:]
print(len(all_reviews))
all_reviews

111092


['Thank you for your help and support!',
 nan,
 'Excellent and professional - highly recommended!',
 'Top illustrator. Superb quality. Fast turn-around. Knows how to translate briefings into visuals. ',
 'What should i say, perfect as always. Thanks :)',
 'Outstanding work, will definitely use again :)',
 'Good provider',
 'excellent work i will work with paul againe',
 'Awesome Awesome Awesome....always fantastic and always will be back',
 'Another great partner',
 'Love working with Blandine! She is a rockstar designer & so professional.',
 'Marcin is the best!',
 'Great design, fantastic job!',
 'The Freelancer helped me with my SEO and set some good backlinks. Will continue to work with him and hopefully get good ranks on Google',
 'we love the design, great work and we will be happy to work with Rali again!!!',
 'Great to work with and made the process and day effortless.',
 'I would recommend JeoFreelance services.',
 'Perfect, thank you. Delivered as promised, A+.',
 'Great desi

## Read in Replacement words

In [5]:
replacement_words_single = pd.read_csv(data_dir+'replacements_singles.csv',index_col=0)
replacement_words_single_no_overlap = pd.read_csv(data_dir+'replacements_minus_flag.csv',index_col=0)

In [6]:
print(replacement_words_single.shape[0])
replacement_words_single.head()

162


,rep_singles
0,kind
1,stable
2,straightforward
3,adapting
4,professional


In [7]:
print(replacement_words_single_no_overlap.shape[0])
replacement_words_single_no_overlap.head()

98


,rep_minus_flag
0,adapting
1,professional
2,compelling
3,observant
4,welcoming


In [8]:
rwsno = list(replacement_words_single_no_overlap.rep_minus_flag)

## Read in Flagged Words

In [9]:
flagged_words_single = pkl.load(open(data_dir + 'flagged_words_single.p','rb'))

## Lemmatize Words Datasets

In [10]:
import nltk
nltk.download('wordnet')
import random
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\atulg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
replacement_words_lemmatized = [wordnet_lemmatizer.lemmatize(w) for w in rwsno]
flagged_words_lemmatized = [wordnet_lemmatizer.lemmatize(w) for w in flagged_words_single]

In [12]:
replacement_words_set = set(replacement_words_lemmatized)
flagged_words_set = set(flagged_words_lemmatized)

## Generate Dataframe

In [13]:
flagged_words_set = [word.lower() for word in flagged_words_set]
replacement_words_set = [word.lower() for word in replacement_words_set]

In [15]:
from nltk import tokenize

columns = ['review','flagged_word','flagged_index','problematic']
df = pd.DataFrame(columns=columns)


flagged_count = 0
replacement_count = 0

for review in tqdm(all_reviews):
    if type(review)==float:
        continue
    review_split = tokenize.sent_tokenize(review)
    review_tokenized = [tokenize.word_tokenize(review_split[i]) for i in range(len(review_split))]
    review_tokenized = [[w.lower() for w in review_sent] for review_sent in review_tokenized] 
    review_lemmatized = [[wordnet_lemmatizer.lemmatize(w) for w in review_sent] for review_sent in review_tokenized] 
    
    for i in range(len(review_lemmatized)):
        for j in range(len(review_lemmatized[i])): 
            if review_lemmatized[i][j] in flagged_words_set:
                entry = {'review': review_tokenized[i],'flagged_word':review_tokenized[i][j],'flagged_index':j,'problematic':1}
                df = df.append(entry,ignore_index=True)
            elif review_lemmatized[i][j]  in replacement_words_set:
                entry = {'review': review_tokenized[i],'flagged_word':review_tokenized[i][j],'flagged_index':j,'problematic':0}
                df = df.append(entry,ignore_index=True)


  1%|▊                                                                          | 1289/111092 [00:07<10:54, 167.86it/s]


  2%|█▊                                                                         | 2601/111092 [00:15<12:45, 141.80it/s]


  4%|██▋                                                                        | 3894/111092 [00:24<13:28, 132.59it/s]


  5%|███▍                                                                       | 5108/111092 [00:32<13:07, 134.60it/s]


  6%|████▏                                                                       | 6170/111092 [00:40<18:14, 95.85it/s]


...

 99%|█████████████████████████████████████████████████████████████████████████▎| 110150/111092 [41:28<00:44, 21.11it/s]


 99%|█████████████████████████████████████████████████████████████████████████▌| 110420/111092 [41:39<00:23, 28.14it/s]


100%|█████████████████████████████████████████████████████████████████████████▋| 110678/111092 [41:51<00:19, 21.39it/s]


100%|█████████████████████████████████████████████████████████████████████████▉| 110932/111092 [42:02<00:06, 24.94it/s]


100%|██████████████████████████████████████████████████████████████████████████| 111092/111092 [42:08<00:00, 27.55it/s]

In [17]:
# write python dict to a file
output = open(data_dir +'master_df.p', 'wb')
pkl.dump(df, output)
output.close()

In [16]:
df

,review,flagged_word,flagged_index,problematic
0,"[thank, you, for, your, help, and, support, !]",support,6,1
1,"[excellent, and, professional, -, highly, recommended, !]",excellent,0,1
2,"[excellent, and, professional, -, highly, recommended, !]",professional,2,0
3,"[superb, quality, .]",superb,0,1
4,"[outstanding, work, ,, will, definitely, use, again, :, )]",outstanding,0,1
5,"[good, provider]",provider,1,1
6,"[excellent, work, i, will, work, with, paul, againe]",excellent,0,1
7,"[another, great, partner]",great,1,1
8,"[she, is, a, rockstar, designer, &, so, professional, .]",professional,7,0
9,"[great, design, ,, fantastic, job, !]",great,0,1


In [18]:
len_reviews = [len(sentence) for sentence in df["review"].to_list()]

In [19]:
import numpy as np
np.max(len_reviews)

174

In [20]:
df[:50]

,review,flagged_word,flagged_index,problematic
0,"[thank, you, for, your, help, and, support, !]",support,6,1
1,"[excellent, and, professional, -, highly, recommended, !]",excellent,0,1
2,"[excellent, and, professional, -, highly, recommended, !]",professional,2,0
3,"[superb, quality, .]",superb,0,1
4,"[outstanding, work, ,, will, definitely, use, again, :, )]",outstanding,0,1
5,"[good, provider]",provider,1,1
6,"[excellent, work, i, will, work, with, paul, againe]",excellent,0,1
7,"[another, great, partner]",great,1,1
8,"[she, is, a, rockstar, designer, &, so, professional, .]",professional,7,0
9,"[great, design, ,, fantastic, job, !]",great,0,1


In [21]:
import matplotlib.pyplot as plt

plt.hist(len_reviews, bins = 10)

(array([1.14031e+05, 2.27360e+04, 2.11900e+03, 3.48000e+02, 7.00000e+01,
        3.80000e+01, 1.00000e+01, 0.00000e+00, 7.00000e+00, 3.00000e+00]),
 array([  1. ,  18.3,  35.6,  52.9,  70.2,  87.5, 104.8, 122.1, 139.4,
        156.7, 174. ]),
 <a list of 10 Patch objects>)

In [22]:
[i for i, j in enumerate(len_reviews) if j == 174]

[45057, 45058, 45059]

In [23]:
df.iloc[45057,0]

['i',
 'hired',
 'this',
 'freelancer',
 'to',
 'tweak',
 'a',
 'current',
 'logo',
 'and',
 'do',
 'a',
 'similar',
 'version',
 'with',
 'different',
 'lettering',
 ',',
 'the',
 'one',
 'to',
 'be',
 'tweaked',
 'was',
 'slightly',
 'distorted',
 'when',
 'blown',
 'up',
 ',',
 'i',
 'thought',
 'he',
 'was',
 'competant',
 'but',
 'generally',
 'hiring',
 'freelancers',
 'on',
 'people',
 'per',
 'hour',
 'is',
 'hit',
 'and',
 'miss',
 ',',
 'this',
 'one',
 'was',
 'miss',
 ',',
 'he',
 'seemed',
 'impressive',
 'at',
 'first',
 ',',
 'the',
 'task',
 'was',
 'very',
 'easy',
 'to',
 'just',
 'align',
 'and',
 'soften',
 'up',
 'a',
 'couple',
 'of',
 'parts',
 'of',
 'the',
 'image',
 'within',
 'my',
 'logo',
 ',',
 '10',
 'minute',
 'job',
 'on',
 'photoshop',
 'or',
 'any',
 'other',
 'design',
 'software',
 ',',
 'once',
 'hired',
 'he',
 'started',
 'to',
 'ask',
 'me',
 'what',
 'font',
 'was',
 'used',
 ',',
 'i',
 'did',
 'not',
 'ask',
 'him',
 'to',
 'touch',
 'the',
 